## 2.4 통계 기반 기법 개선하기
앞 절에서 단어의 동시발생 행렬을 통해 단어를 벡터로 만들었으나, 이것 개선해보자

### 2.4.1 상호정보량
동시발생 행렬의 원소는 두 단어가 동시에 발생한 횟수를 나타낸다.  
하지만 '발생'횟수는 좋은 특징이 이낟. 고빈도 단어를 보면 알수 있는데 말뭉치에서 'the'나 'car'의 동시발생을 보면 두 단어는 동시발생확률이 매우 높다. 하지만 the car보다는 car drive가 더 관련성이 강하다.   
이 문제를 해결하기 위해서 `점별 상호정보량(PMI)`라는 척도를 사용한다.

$$PMI(x,y)=log_2 \frac{P(x,y)}{P(x)P(y)}$$

PMI를 통하면 아까와 같은 문제를 해결할 수 있다.  
전체 말뭉치의 단어수(N)이 10,000일 때, 'the'가 1000번, 'car'가 20번, 'drive'가 10번 발생했고, the car는 10번 동시발생, car drive는 5회라고 가정한다.
이 때, the 와 car의 PMI는 2.32, car와 drive의 PMI는 7.97로 원하는 척도를 얻을 수 있다.

PMI에 한 가지 문제가 있는데, 바로 두 단어의 동시발생 횟수가 0이면 $log_20 = - \infty$ 가 된다는 점이다. 이 문제를 피하기 위해 실제로 구현할 때에는 양의 상호정보량(PPMI: Positive PMI)를 사용한다.

$$PPMI(x,y)=max(0, PMI(x,y))$$

In [1]:
import numpy as np

def ppmi(C, verbose=False, eps=1e-8) -> list:
    """Make ppmi from 동시발생 행렬

    Args:
        C (list): 동시발생행렬
        verbose (bool, optional): 상세히 출력하기 위한 옵션. Defaults to False.
        eps (float, optional): log2가 음의 무한대가 되는 것을 막기 위한 임의의 작은 수. Defaults to 1e-8.

    Returns:
        list: ppmi
    """
    M = np.zeros_like(C, dtype=np.float32)
    N = np.sum(C)
    S = np.sum(C, axis=0)
    total = C.shape[0] * C.shape[1]
    cnt = 0

    for i in range(C.shape[0]):
        for j in range(C.shape[1]):
            pmi = np.log2(C[i, j] * N / (S[j] * S[i]) + eps)
            M[i, j] = max(0, pmi)

            if verbose:
                cnt += 1
                if cnt % (total//100 + 1) == 0:
                    print(f"{100*cnt/total:.1f} 완료")
    return M

In [2]:
import numpy as np
from utils import preprocess, create_co_matrix, cos_similarity, ppmi

text = 'You say goodbye and I say hello.'
corpus, word_to_id, id_to_word = preprocess(text)
vocab_size = len(word_to_id)
C = create_co_matrix(corpus, vocab_size)
W = ppmi(C)

np.set_printoptions(precision=3)
print('동시발생 행렬')
print(C)
print('-'*50)
print('PPMI')
print(W)

동시발생 행렬
[[0 1 0 0 0 0 0]
 [1 0 1 0 1 1 0]
 [0 1 0 1 0 0 0]
 [0 0 1 0 1 0 0]
 [0 1 0 1 0 0 0]
 [0 1 0 0 0 0 1]
 [0 0 0 0 0 1 0]]
--------------------------------------------------
PPMI
[[0.    1.807 0.    0.    0.    0.    0.   ]
 [1.807 0.    0.807 0.    0.807 0.807 0.   ]
 [0.    0.807 0.    1.807 0.    0.    0.   ]
 [0.    0.    1.807 0.    1.807 0.    0.   ]
 [0.    0.807 0.    1.807 0.    0.    0.   ]
 [0.    0.807 0.    0.    0.    0.    2.807]
 [0.    0.    0.    0.    0.    2.807 0.   ]]


동시발생을 PPMI 행렬로 변환해보았다. PPMI 행렬의 각 원소는 모두 0 이상의 실수이다.  
이제 우리는 더 좋은 척도로 이뤄진 행렬(더 좋은 단어 벡터)을 손에 쥐었습니다.

그러나 PPMI에도 여전히 문제가 있다. 말뭉치의 어휘수가 증가함에 따라 각 단어벡터의 차원 수도 증가한다는 문제다. 예를 들어 말뭉치의 어휘 수가 10만 개라면 그 벡터의 차원수도 똑같이 10만이 된다. 10만 차원의 벡터를 다룬다는 것은 그다지 현실적이지 않다.
이 문제에 대처하고자 자주 수행하는 기법이 바로 벡터의 차원 감소이다.

### 2.4.2 차원 감소
차원감소는 문자 그대로 벡터의 차원을 줄이는 방법을 말한다. 그러나 단순히 줄이는 것이 아니라 '중요한 정보'는 최대한 유지하면서 줄이는 게 핵심이다.  
아래 사진 처럼 데이터의 분포를 고려해 중요한 '축'을 찾는 일을 말한다.

![](https://github.com/yesinkim/Deep-Learning-From-Scratch2/blob/main/deep_learning_2_images/fig%202-8.png?raw=true)
왼쪽은 데이터점들을 2차원 좌표에 표시한 모습이고, 오른쪽은 새로운 축을 도입하여 똑같은 데이터를 좌표축 하나만으로 표시했다. 이 때 각 데이터점의 값은 새로운 축으로 사영된 값으로 변한다. 여기서 중요한 것은 가장 적합한 축을 찾아내는 일로, 1차원 값만으로도 데이터의 본질적인 차이를 구별할 수 있어야 한다.

> NOTE: 원소 대부분이 0인 행렬 또는 벡터를 희소(Sparse)행렬 및 희소벡터라고 한다. 차원 감소의 핵심은 희소벡터에서 중요한 축을 찾아내 더 적은 밀집벡터로 변환하는 것이다. 이 조밀한 벡터야말로 우리가 원하는 단어의 분산표현이다.


#### SVD(특잇값 분해)

차원을 감소시키는 방법은 여러 가지이지만, 우리는 **특잇값분해(SVD:Singular Value Decomposition)** 를 이용한다.  
SVD는 임의의 행렬을 세 행렬의 곱으로 분해하며, 수식으로는 다음과 같다
$$X = USV^T$$
SVD는 임의의 행렬 X를 U, S, V라는 세 행렬의 곱으로 분해한다.
U와 V는 직교행렬이고, 그 열 벡터는 서로 직교한다. 또한 S는 대각행렬(대각성분 외에는 모두 0인 행렬)이다. (??뭔말)
![](https://github.com/yesinkim/Deep-Learning-From-Scratch2/blob/main/deep_learning_2_images/fig%202-9.png?raw=true)
$U$는 직교행렬이다. 이 직교행렬은 어떠한 축(기저)을 형성한다. 지금 우리의 맥락에서는 이 $U$행렬을 '단어 공간'으로 취급할 수 있고, $S$는 대각행렬로 '특잇값'이 큰 순서로 나열되어 있다. 특잇값이란 '해당 축'의 중요도라고 간주할 수 있다. 아래 사진처럼 중요도가 낮은 원소를 깎아내는 방법을 생각할 수 있다.
![](https://github.com/yesinkim/Deep-Learning-From-Scratch2/blob/main/deep_learning_2_images/fig%202-10.png?raw=true)

In [3]:
from utils import preprocess, create_co_matrix, ppmi

text = 'You say goodbye and I say hello.'
corpus, word_to_id, id_to_word = preprocess(text)
vocab_size = len(id_to_word)
C = create_co_matrix(corpus, vocab_size, window_size=1)
W = ppmi(C)

# SVD
U, S, V = np.linalg.svd(W)
print(C[0])     # 동시발생 행렬
print(W[0])     # PPMI 행렬
print(U[0])     # SVD

[0 1 0 0 0 0 0]
[0.    1.807 0.    0.    0.    0.    0.   ]
[-3.409e-01 -1.110e-16 -4.441e-16  1.205e-01  9.323e-01  0.000e+00
  3.207e-16]


이 결과에서 보듯 원래는 희소벡터는 W[0]이 SVD를 통해 밀집벡터 U[0]로 변했다. 그리고 이 밀집벡터의 차원을 감소시키려면, 단순히 처음의 두 원소를 꺼내면 된다(중요한 순서로 정렬되어 있기 때문에)

In [4]:
print(U[0, :2])     # 쫌 이상함

[-3.409e-01 -1.110e-16]


In [6]:
# # 각 단어를 2차원 벡터로 표편한 후 그래프로 그려본다        --> 안나타나는데..
# import matplotlib.pyplot as plt

# for word, word_id in word_to_id.items():
#     plt.annotate(word, U[word_id, 0], U[word_id, 1])

# plt.scatter(U[:,0], U[:,1], alpha=0.5)
# plt.show

![](https://github.com/yesinkim/Deep-Learning-From-Scratch2/blob/main/deep_learning_2_images/fig%202-11.png?raw=true)
이 그림을 보면 'goodbye'와 'hello', 그리고 'you'와 'i'가 제법 가까이 있어 우리의 직관과 비슷해진 것을 볼 수 있다.
하지만 너무 작은 말뭉치를 사용했기 때문에 결과가 석연치 않다. PTB데이터 셋이라는 더 큰 말뭉치를 이용해서 똑같은 실험을 진행해보자.

> WARNING: 행렬의 크기가 N이면 SVD 계산은 O(N^3)이 걸린다. 이는 현실적으로 감당하기 어려운 수준이므로 Truncated SVD같은 더 빠른 기법을 이용한다. 이것은 특잇값이 작은 것은 버리는 방식으로 성능 향상을 꾀한다. 다음 절에서도 skicit-learn 의 truncated SVD를 이용한다.

### 2.4.4 PTD 데이터 셋

In [7]:
import os
import sys
import ptb

corpus, word_to_id, id_to_word = ptb.load_data('train')

print('말뭉치 크기:', len(corpus))
print('corpus[:30]:', corpus[:30])
print()
print('id_to_word[0]:', id_to_word[0])
print('id_to_word[1]:', id_to_word[1])
print('id_to_word[2]:', id_to_word[2])
print()
print("word_to_id['car']:", word_to_id['car'])
print("word_to_id['happy']:", word_to_id['happy'])
print("word_to_id['lexus']:", word_to_id['lexus'])


말뭉치 크기: 929589
corpus[:30]: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29]

id_to_word[0]: aer
id_to_word[1]: banknote
id_to_word[2]: berlitz

word_to_id['car']: 3856
word_to_id['happy']: 4428
word_to_id['lexus']: 7426


### 2.4.5 PTB 데이터셋 평가

In [8]:
from utils import create_co_matrix, most_similar
window_size = 2
wordvec_size = 100

corpus, word_to_id, id_to_word = ptb.load_data('train')
vocab_size = len(word_to_id)
print('동시발생 수 계산')
C = create_co_matrix(corpus, vocab_size, window_size)
print('PPMI 계산')  # 이런거 왜 쓰는지?ㅋㅋ
W = ppmi(C, verbose=True)

print('SVD 계산')
try: 
    # truncate SVD
    from sklearn.utils.extmath import randomized_svd
    U, S, V = randomized_svd(W, n_components=wordvec_size, n_iter=5,
                             random_state=None)
except ImportError:
    U, S, V = np.linalg.svd(W)

word_vecs = U[:, :wordvec_size]

querys = ['you', 'year', 'car', 'toyota']
for query in querys:
    most_similar(query, word_to_id, id_to_word, word_vecs, top=5)

동시발생 수 계산
PPMI 계산
1.0 완료
2.0 완료
3.0 완료
4.0 완료
5.0 완료
6.0 완료
7.0 완료
8.0 완료
9.0 완료
10.0 완료
11.0 완료
12.0 완료
13.0 완료
14.0 완료
15.0 완료
16.0 완료
17.0 완료
18.0 완료
19.0 완료
20.0 완료
21.0 완료
22.0 완료
23.0 완료
24.0 완료
25.0 완료
26.0 완료
27.0 완료
28.0 완료
29.0 완료
30.0 완료
31.0 완료
32.0 완료
33.0 완료
34.0 완료
35.0 완료
36.0 완료
37.0 완료
38.0 완료
39.0 완료
40.0 완료
41.0 완료
42.0 완료
43.0 완료
44.0 완료
45.0 완료
46.0 완료
47.0 완료
48.0 완료
49.0 완료
50.0 완료
51.0 완료
52.0 완료
53.0 완료
54.0 완료
55.0 완료
56.0 완료
57.0 완료
58.0 완료
59.0 완료
60.0 완료
61.0 완료
62.0 완료
63.0 완료
64.0 완료
65.0 완료
66.0 완료
67.0 완료
68.0 완료
69.0 완료
70.0 완료
71.0 완료
72.0 완료
73.0 완료
74.0 완료
75.0 완료
76.0 완료
77.0 완료
78.0 완료
79.0 완료
80.0 완료
81.0 완료
82.0 완료
83.0 완료
84.0 완료
85.0 완료
86.0 완료
87.0 완료
